In [29]:
import re
from collections import Counter, defaultdict
from typing import List, Tuple
import os
import random
import numpy as np
import re
import math
from itertools import product

In [ ]:
class ngram:
    def __init__(self, n : int, korpus : str, smoothing = False):
        self.n                          = n
        self.smoothing                  = smoothing
        self.toks                       = self.init_toks(korpus)

        self.vocab, self.vocab_count    = self.init_vocab()

        self.counts                     = self.count()


    def init_toks(self, korpus) -> List[List[str]]:
        # Alle Interpunktionszeichen außer \w \s ; . ? ! entfernen
        text_no_p = re.sub(r"[^\w\s;.?!]", "", korpus)
        # Leerzeichen vor jedem übrigen Punktionszeichen setzen
        text_no_p = re.sub(r"([;.?!])", r" \1", text_no_p)

        # Satzgrenzen
        text_no_p_ends = re.sub(r"([?!;.])", r"\1 </s>|||<s> ", text_no_p)
        text_no_p_ends = "<s> " + text_no_p_ends + " </s>"

        text_no_p_ends = text_no_p_ends.lower()
        
        text_no_p_ends_list = re.findall(r'</s>|<s>|\w+|[;.!?]', text_no_p_ends)

        toks = []
        group = []

        for tok in text_no_p_ends_list:
            group.append(tok)

        if tok == "</s>":
            toks.append(group)
            group = []


        return toks
    
    def random_unk(self):
        for i, num in enumerate(self.vocab_count):
            if num == 1:
                start_idx = i
                break

        choices = []

        idx_range = range(start_idx, len(self.vocab_count))
        for _ in range(10):
            rnd_idx = random.choices(idx_range)
            choices.append(rnd_idx)
        
        for i in rnd_idx:
            for j, sent in enumerate(self.toks):
                for k, w in enumerate(sent):
                    if w == self.vocab[i]:
                        self.toks[j][k] = "<UNK>"
            
            self.vocab[i] = "<UNK>"
        
        

    def init_vocab(self):
        counter = Counter(tok for sentence in self.toks for tok in sentence)
        
        items = counter.most_common()    # Liste von (token, count)
        
        vocab, vocab_count = zip(*items)
        
        return list(vocab), list(vocab_count)


    def count(self):
        """
        counts: Dict[ Tuple(context), Counter(next_word → count) ]
        """ # Counter([1,1,1]) => "Ich habe": {1: 3}
        counts = defaultdict(Counter)
        for sent in self.toks:
            for i in range(len(sent) - self.n + 1):
                ctx  = tuple(sent[i:i + (self.n - 1)])
                nxt  = sent[i + self.n - 1]
                counts[ctx][nxt] += 1

        if self.smoothing:
            # Add-one Smoothing: für jeden Kontext und jede Vokabel +1
            for ctx in counts:
                for w in self.vocab:
                    counts[ctx][w] += 1

        return counts

    def next_word(self, seed):
        """
        Gibt genau ein Wort zurück
        """
        toks = seed.lower().split(" ")
        # Padding mit <s>
        while len(toks) < self.n - 1:
            toks.insert(0, "<s>")
        
        ctx = tuple(toks[-(self.n - 1):])

        # Hol den Counter für diesen Kontext
        counter = self.counts.get(ctx, None)
        if not counter:
            # unbekannter Kontext => Nehme random Wort aus dem Vokabular
            return random.choice(self.vocab)
        
        words, weights = zip(*counter.items())

        # Nächstes Wort
        return random.choices(words, weights=weights, k=1)[0]

    def generate(self, seed: str, length: int) -> str:
        toks = seed.lower().split()
        # Padding links mit <s>
        while self.n > 1 and len(toks) < self.n - 1:
            toks.insert(0, "<s>")

        for _ in range(length):
            nxt = self.next_word(" ".join(toks[-(self.n - 1):]))
            toks.append(nxt)
            if nxt == "</s>":
                # toks.append(nxt)
                break

        # Gib den Text zurück
        return " ".join(toks)
    
    def len_grams(self):
        if self.n == 1:
            return len(self.counts.get(()))
        
        else:
            return sum(len(counter) for counter in self.counts if counter)
        
    
    def ppx(self, test_set : str) -> float:
        test_set = self.init_toks(test_set)
        
        N = sum(
            max(0, len(sent) - self.n + 1)
            for sent in test_set
        )

        V = len(self.vocab)

        log_sum = 0.0
        for sent in test_set:
            for i in range(len(sent) - self.n + 1):
                ctx = tuple(sent[i:i + (self.n - 1)])
                nxt = sent[i + self.n - 1]

                # Falls Kontext nicht vorhanden -> 0
                count_ctx_w = self.counts.get(ctx, Counter()).get(nxt, 0)
                count_ctx = sum(self.counts.get(ctx, Counter()).values())

                # Laplace add one
                num = count_ctx_w + 1
                # Vokabular addieren
                denom = count_ctx + V              

                prob = num / denom
                log_sum += -math.log(prob)
            

        return math.exp(log_sum / N)

In [32]:
raw_text = ""

for txt in os.listdir("korpus"):
    with open(f"korpus/{txt}", "r", encoding="utf-8") as f:
        content = f.read()

    raw_text += content + " "

## Modelle trainieren

In [33]:
LM1 = ngram(1, korpus=raw_text, smoothing=False)

LM2A = ngram(2, korpus=raw_text, smoothing=False)
LM2B = ngram(2, korpus=raw_text, smoothing=True)

LM3A = ngram(3, korpus=raw_text, smoothing=False)
# LM3B = ngram(3, korpus=raw_text, smoothing=True)

## LM Statistik

In [52]:
print(f"|V|: {len(LM1.vocab)}") ; print()

print("--------Language Model 1--------")
print(f"|Uni|: {LM1.len_grams()}") ; print()

print("--------Language Model 2--------")
print(f"|Bi|a: {LM2A.len_grams()}")
ratio = LM2A.len_grams() / (len(LM2A.vocab) ** 2)
print(f"Anteil nicht-null: {ratio*100:2f}%") ; print()
print(f"|Bi|b: {LM2B.len_grams()}")

print("--------Language Model 3--------")
print(f"|Tri|a: {LM3A.len_grams()}")
ratio = LM3A.len_grams() / (len(LM3A.vocab) ** 3)
print(f"Anteil nicht-null: {ratio*100:2f}%") ; print()

# print(f"|Tri|b: {sum_tri_LM3B}")

|V|: 9339

--------Language Model 1--------
|Uni|: 9339

--------Language Model 2--------
|Bi|a: 9339
Anteil nicht-null: 0.010708%

|Bi|b: 9339
--------Language Model 3--------
|Tri|a: 102892
Anteil nicht-null: 0.000013%



## Text generierung

In [54]:
LM1.generate("<s>", 10)

'<s> diener hereinzutreten sünde zuge zerkratzen fast erkundigte d zurücksank auspacken'

In [40]:
LM2B.generate("<s> es war einmal", 10)

'<s> es war einmal willigte ausflug erbschaft seid dahinging sogleich leute ganz gewiß gewünscht'

In [70]:
LM3A.generate("<s> es war einmal", 10)

'<s> es war einmal mitten im zimmer ; </s>'

In [63]:
LM3A.counts.get(("war", "einmal"))

Counter({'ein': 4, 'eine': 2, 'gemacht': 1, 'mitten': 1})

## Text Klassifikation

In [45]:
text1 = """Es war einmal eine Königin. Anspruchsvoll war sie und nicht sehr schön, doch weil 
sie ihren Untertanen Süße und Glück versprach, schenkten sie ihr die besten Böden. Das 
Reich der Königin wuchs, aber bald wurde sie krank, und ihre Feinde nutzten ihre 
Schwäche. Unruhe regte sich im Volk, die Zweifel nahmen zu. Wie ist die Königin zu 
retten? Oder wäre es besser, sie sterben zu lassen?"""

In [46]:
text2 = """So ließe sie sich erzählen, die Geschichte der runzligen Zuckerrübe, der "Königin der 
Feldfrüchte", wie Bauern sie nennen. Märchenhaft war sowohl ihr Aufstieg in Deutschland und anderen Ländern Europas als auch der Wohlstand, den sie manchem Landwirt 
bescherte. Der aus ihr gewonnene Zucker verfeinerte eine Fülle von Lebensmitteln, kaum 
etwas kam noch ohne ihn aus. Inzwischen jedoch hat Zucker in unserer Nahrung ein 
solches Übermaß erreicht, dass er mehr Schaden als Beglückung verspricht. Zu viel Zucker 
kann zu Übergewicht führen, zu Diabetes, Herz-Kreislauf-Erkrankungen, Bluthochdruck. 
Selbst zwischen Zuckerkonsum und Krebs sehen einige Wissenschaftler Zusammenhänge."""

In [71]:
print(LM1.ppx(text1))
print(LM1.ppx(text2))

500.06876648965954
1314.3432727520342


In [50]:
LM2A.ppx(text1)

1407.8475803904273